In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy.optimize import curve_fit
import glob
import os
import sys
import matplotlib.backends.backend_pdf
import matplotlib.gridspec as gridspec
from scipy.stats import norm
from scipy.stats import lognorm
from scipy import linalg
import pprint
import re
plt.rc('font', family='serif')
from matplotlib import rc
import pandas as pd
from astropy import constants as const
from astropy import units as units
from IPython.display import Markdown as md
import sympy
import pandas
from sympy import integrate, diff, sqrt, cos, sin, pi, exp, log
from sympy.abc import * 
i = sqrt(-1)
import numpy as np
import sympy.printing as printing
latp = printing.latex
hbar = sympy.symbols("hbar", real=True)
import mpmath 
import plotly.express as px
import plotly.graph_objs as go
hbar

hbar

In [2]:
from error_propagation import *

2.1459505607619285
3.0730730555044836
3.3370598254672865
(a + 2)/(a + b*cos(pi*c/45))
(3.50656581341894, 1.8267621151828408)


The above output is correct

In [3]:
raw_data = pd.read_csv(glob.os.getcwd() + "\\ay105lab3.csv")
raw_data

,Unnamed: 0,Part 2: Wavelength of Laser,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Count of min (\pm sqrt) [LEFT],Count of min (\pm sqrt) [RIGHT],\Delta s \pm .5 um,"\n+x towards wall on left when walking in,+y a...",NaN,NaN
1,NaN,30,30,10.5,NaN,after the first measurement we moved the scree...,NaN
2,NaN,34,34,10.5,NaN,every delta s should be 10.5,NaN
3,NaN,35,35,11.5,NaN,red laser!,in reality it is green
4,NaN,50,51?,16.5,NaN,error in counts is sqrt n because standard dev...,NaN
5,NaN,50,51?,17,NaN,make sure phase shift is in the right units an...,NaN
6,NaN,47,49,17,NaN,jonah counted 49 for the right projection,NaN
7,NaN,80,80,22.5,NaN,note: limited by the setup. the last measureme...,NaN
8,trust the arcminutes!,NaN,NaN,NaN,NaN,counting fringes corresponds to change in path...,NaN
9,"start angle (degrees, arcmin if known)",Part III: Refractive Index,using thorlabs cr1 rotational stage,\pm 1 because human eyeballs and small tick marks,NaN,lots of realigning. Jonah and Carlos have pict...,NaN


# Part 2

In [4]:
part2_df = raw_data.iloc[1:8,1:4]
#raw_data.iloc[0, 5:] = ["comments" for x in range(raw_data.shape[1]-5)]
part2_df.columns = raw_data.iloc[0,1:4]
part2_df

,Count of min (\pm sqrt) [LEFT],Count of min (\pm sqrt) [RIGHT],\Delta s \pm .5 um
1,30,30,10.5
2,34,34,10.5
3,35,35,11.5
4,50,51?,16.5
5,50,51?,17
6,47,49,17
7,80,80,22.5


In [5]:
if object in [x for x in part2_df.dtypes]:    
    part2_df.iloc[3, 1], part2_df.iloc[4, 1] = part2_df.iloc[3, 1][:-1], part2_df.iloc[4, 1][:-1]
for x in part2_df:
    if "Count" in x or "count" in x:
        part2_df[x] = part2_df[x].astype(int)
    else:
        part2_df[x] = part2_df[x].astype(float)
part2_df

,Count of min (\pm sqrt) [LEFT],Count of min (\pm sqrt) [RIGHT],\Delta s \pm .5 um
1,30,30,10.5
2,34,34,10.5
3,35,35,11.5
4,50,51,16.5
5,50,51,17.0
6,47,49,17.0
7,80,80,22.5


Recall from the notes that 
\begin{align}
\Delta\phi = \frac{2\pi}{\lambda}\Delta x
\end{align}
where $\Delta\phi$ is the change in phase, $\lambda$ is the wavelength, and $\Delta s$ is the difference in path length. Rearranging and solving for the wavelength yields
\begin{align}
\lambda = \frac{2\pi}{\Delta\phi}\Delta x
\end{align}
For minima we have $\Delta\phi=(2n+1)\phi$ and for maxima we have  $\Delta\pi=2n\pi$ where $n\in\mathbb{Z}$ in order for the minima and maxima to correspond to complete destructive and constructive interference, respectively. Since we counted minimum we have $\Delta\phi=(2n+1)\pi$. Substituting this phase difference yields
\begin{align}
\lambda = \frac{2}{2n+1}\Delta x
\end{align}
For the zeroeth order mode we have 
\begin{align}
\lambda = 2\Delta x
\end{align}
Since we know that $\frac{\Delta s}{N} = \Delta x$ we have
\begin{align}\label{eq:part2}
\lambda = 2\frac{\Delta s}{N}
\end{align}

In [6]:
part2_df["Wavelength (𝜇m)"] = (4 * part2_df["\Delta s \pm .5 um"] / 
    (part2_df["Count of min (\pm sqrt) [LEFT]"] + 
     part2_df["Count of min (\pm sqrt) [RIGHT]"]))
part2_df

,Count of min (\pm sqrt) [LEFT],Count of min (\pm sqrt) [RIGHT],\Delta s \pm .5 um,Wavelength (𝜇m)
1,30,30,10.5,0.700000
2,34,34,10.5,0.617647
3,35,35,11.5,0.657143
4,50,51,16.5,0.653465
5,50,51,17.0,0.673267
6,47,49,17.0,0.708333
7,80,80,22.5,0.562500


## Calculation Part 2

In [7]:
final_wavelength = np.average(part2_df.iloc[:,-1])
md(latp(np.round(final_wavelength, 2)) + " 𝜇m")

0.65 𝜇m

Now to find the average number of minima and the corresponding uncertainty. Since we were simply counting, the uncertainty is 1 over the square root of the counts. 

In [8]:
fun = ( a + b ) / 2
counts_averages, counts_uncertainties = [], []
row_list = range(part2_df.shape[0])
for row in row_list:
    left, right = part2_df.iloc[row, 0], part2_df.iloc[row, 1]
    expct, unc = general_with_arrays([left, right], [1/np.sqrt(left), 1/np.sqrt(right)], fun)
    counts_averages.append(expct)
    counts_uncertainties.append(unc)
counts_averages, counts_uncertainties

([30.0, 34.0, 35.0, 50.5, 50.5, 48.0, 80.0],
 [0.12909944487358055,
  0.12126781251816647,
  0.11952286093343935,
  0.09950859653474027,
  0.09950859653474027,
  0.1020842287157852,
  0.07905694150420949])

Since we averaged the number of minima observed from each interference pattern we now use equation \ref{eq:part2}

Note that the function named general_with_arrays uses the following the find the best estimate: 

lambdify(symbol_list[:2 * len(row_list)], fun)(*symbol_list[:2 * len(row_list)])


In [9]:
fun = 0 
for row in row_list:
    fun += 2*(symbol_list[row] / symbol_list[row + len(row_list)]) / (len(row_list))
fun

2*a/(7*h) + 2*b/(7*i) + 2*c/(7*j) + 2*d/(7*k) + 2*e/(7*l) + 2*f/(7*m) + 2*g/(7*n)

In alphabetical order, a through g are the values for $\Delta s$, whereas h through n are the values for $N$, and this sum is the expectation value of the wavelength of the LASER.

In [10]:
part2_expectation_inputs = [x for x in part2_df["\Delta s \pm .5 um"]]
part2_uncertainty_inputs = len(row_list) * [.5]
for row in row_list:
    part2_expectation_inputs.append(counts_averages[row])
    part2_uncertainty_inputs.append(counts_uncertainties[row])
part2_wavelength, part2_unc = general_with_arrays(part2_expectation_inputs, part2_uncertainty_inputs, fun)
md("Thus we have for the wavelength \\begin{equation} \\lambda = " 
   + latp(np.round(part2_wavelength, 2)) + "\\pm " + latp(np.round(part2_unc, 3)) + " \\text{ 𝜇m} "
  + "\\end{equation}")

Thus we have for the wavelength \begin{equation} \lambda = 0.65\pm 0.009 \text{ 𝜇m} \end{equation}

# Part 3
Recall from the lab handout we have for the index of refraction $n$
\begin{align}
b \equiv \frac{N\lambda}{2t} \quad\ n = \frac{(b+\cos(\alpha) - 1)^2 + \sin^2\alpha}{2(-b-\cos(\alpha)+1)}
\end{align}

In [11]:
part3_df = raw_data.iloc[11:15, :5]
part3_df

,Unnamed: 0,Part 2: Wavelength of Laser,Unnamed: 2,Unnamed: 3,Unnamed: 4
11,0,8,8,2,?
12,-0.5,13,14-15,3,25
13,"-0.5, -10",15,15,2,20
14,"0, -10",17,17,3,35


The first column is the reference angle in degree, arcminutes

The second column is the number of minima counted on the left

The third column is the number of minima counted on the right

The fourth column is the final angle in degrees

The fifth column is the final angle's arcminute measurement

## Error Progation and Angle Estimation

In [23]:
part3_df.columns = ["Ref_angle_deg_arcmin", "left_counts", "right_counts", "final_angle_deg", 
                   "final_angle_arcmin"]
part3_df.iloc[1, 2], part3_df.iloc[0, -1] = 14, 30 
part3_df.iloc[0, 0], part3_df.iloc[1, 0] = "0, 0", "-.5, 0" 
part3_df

,Ref_angle_deg_arcmin,left_counts,right_counts,final_angle_deg,final_angle_arcmin
11,"0, 0",8,8,2,30
12,"-.5, 0",13,14,3,25
13,"-0.5, -10",15,15,2,20
14,"0, -10",17,17,3,35


Since each of those angle measurements was read by eye from a scale, each of them has an uncertainty of 0.5, with the exception of the final_angle_arcminute measurement in the 11th row, since we did not yet know to note this value, so it will have a value of $30\pm30$ arcminutes.

In [26]:
fun = pi * ((a+(b/60)) - (c+(d/60))) / 180
angle_expct_arr = []
angle_unc_arr = []
for row in part3_df.index:
    prelim_unc_angle_arr = [.5, .5, .5, .5]
    if row==11 or row==12:
        prelim_unc_angle_arr = [.5, 30, .5, 30]
    ref_angle_deg, ref_angle_arcmin = [float(x) for x in part3_df.loc[row, 
                                            "Ref_angle_deg_arcmin"].split(",")]
    end_angle_deg = float(part3_df.loc[row, "final_angle_deg"]) 
    end_angle_arcmin = float(part3_df.loc[row, "final_angle_arcmin"])
    angle_expct, angle_unc= general_with_arrays(
        [end_angle_deg, end_angle_arcmin, ref_angle_deg, ref_angle_arcmin],
        prelim_unc_angle_arr, fun)
    angle_expct_arr.append(angle_expct)
    angle_unc_arr.append(angle_unc)
part3_df["Angle_Radians"] = angle_expct_arr
part3_df["Angle_Uncertainty_Radians"] = angle_unc_arr
part3_df.iloc[:,-2:]

,Angle_Radians,Angle_Uncertainty_Radians
11,0.043633,0.017453
12,0.068359,0.017453
13,0.052360,0.012343
14,0.065450,0.012343


In [ ]:
my_factor = c *  (d + e) / (4*f)
fun = ((my_factor + cos(a-b) - 1)**2 + sin(a-b)**2) / (2*(-my_factor - cos(a-b) + 1))
fun

In [ ]:
lambdify(symbol_list[:2 * len(row_list)], fun)(*symbol_list[:2 * len(row_list)])

For the sake of time, only the last measurement will be used because it has the highest number of counts and thus the highest resolution. Ideally, all of the measurements would be taken into account in order to achieve the lowest uncertainty possible. This would be done in a manner similar to the analysis for part 2, and would involve using the newly improved error propagation python script. 

In [ ]:
def estimate_n(row):
    left_counts, right_counts = np.float64(part3_df.iloc[-1, 1]), np.float64(part3_df.iloc[-1, 1])
    thickness = 8000 #micron, checked with collaborators
    ref_angle = pi * (np.float64(part3_df.iloc[-1, 0].split(",")[0]) 
                 + (np.float64(part3_df.iloc[-1, 0].split(",")[1])/60) ) / 180
    final_angle = pi * ( 
                    np.float64(part3_df.iloc[-1, 3]) + (np.float64(part3_df.iloc[-1, 4][1])/60) ) / 180
    expct_inputs = [ref_angle, final_angle, part2_wavelength, left_counts, right_counts, thickness]
    fun.subs([(d, left_counts), (e, right_counts), (f , thickness), (a, ref_angle), (b, final_angle),
             (c, part2_wavelength)])  

In [ ]:
fun

In [ ]:
sympy.N(cos(90))